# Google Gen AI SDK Experimentation
***

## SDK Documentation

**GitHub**: https://github.com/google-gemini/generative-ai-python


**Official Docs**: https://ai.google.dev/gemini-api/docs

## Setup

In [3]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file in the current directory
load_dotenv()

True

In [4]:
from google import genai

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

## Generate content

In [5]:
response = client.models.generate_content(
    model='gemini-2.0-flash', contents='How does RLHF work? Keep it short.'
)
print(response.text)

RLHF (Reinforcement Learning from Human Feedback) improves AI by:

1.  **Training a model:** Train a basic language model.
2.  **Collecting human preferences:** Humans rank different model outputs for the same prompt.
3.  **Training a reward model:** Use human rankings to train a reward model that predicts human preferences.
4.  **Fine-tuning with RL:** Use reinforcement learning to further train the original language model, optimizing it to maximize the reward model's score. This aligns the model with human preferences.



## Count Tokens

In [6]:
response = client.models.count_tokens(
    model='gemini-2.0-flash',
    contents='The quick brown fox jumps over the lazy dog.',
)

In [7]:
response

CountTokensResponse(total_tokens=11, cached_content_token_count=None)

In [8]:
response.total_tokens

11

## Structured Output

https://ai.google.dev/gemini-api/docs/structured-output?lang=python

https://github.com/google-gemini/generative-ai-python?tab=readme-ov-file#json-response

Using `genai.types.Schema` (can also pass equivalent `dict`)

In [101]:
# import base64
# import os
# from google import genai
# from google.genai import types


# def generate():
#     client = genai.Client(
#         api_key=os.environ.get("GEMINI_API_KEY"),
#     )

#     model = "gemini-2.0-flash"
#     contents = [
#         types.Content(
#             role="user",
#             parts=[
#                 types.Part.from_text(
#                     text="Provide a detailed recipe for a delicious pasta dish"
#                 ),
#             ],
#         ),
#     ]
#     generate_content_config = types.GenerateContentConfig(
#         temperature=1,
#         top_p=0.95,
#         top_k=40,
#         max_output_tokens=8192,
#         response_mime_type="application/json",
#         response_schema=genai.types.Schema(
#             type=genai.types.Type.OBJECT,
#             enum=[],
#             required=["summary", "instructions"],
#             properties={
#                 "summary": genai.types.Schema(
#                     type=genai.types.Type.STRING,
#                 ),
#                 "instructions": genai.types.Schema(
#                     type=genai.types.Type.ARRAY,
#                     items=genai.types.Schema(
#                         type=genai.types.Type.STRING,
#                     ),
#                 ),
#             },
#         ),
#     )

#     # Uncomment the streaming version if needed (does not work properly in a notebook environment):
#     # for chunk in client.models.generate_content_stream(
#     #     model=model,
#     #     contents=contents,
#     #     config=generate_content_config,
#     # ):
#     #     print(chunk.text, end="")

#     # Use the non-streaming API call instead:
#     response = client.models.generate_content(
#         model=model,
#         contents=contents,
#         config=generate_content_config,
#     )
#     # print(response.text)
#     return response.text

Using *Pydantic* schemas

In [102]:
from pydantic import BaseModel
from typing import List

# Example Pydantic schema for reference
# -------------------------------------
class RecipeResponse(BaseModel):
    title: str
    summary: str
    instructions: List[str]


# Example with optional field for reference
# -----------------------------------------
# from typing import Optional

# class RecipeResponse(BaseModel):
#     title: Optional[str] = None  # now optional
#     summary: str
#     instructions: List[str]


In [103]:
import base64
import os
from google import genai
from google.genai import types

def generate():
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.0-flash"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(
                    text="Provide a detailed recipe for a delicious chicken dish"
                ),
            ],
        ),
    ]
    # Use the Pydantic schema directly in response_schema.
    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        top_k=40,
        max_output_tokens=8192,
        response_mime_type="application/json",
        response_schema=RecipeResponse,
    )

    # Use the non-streaming API call.
    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,
    )
    return response

In [104]:
result = generate()

In [137]:
type(result)

google.genai.types.GenerateContentResponse

`result.text` is a raw JSON string—you get exactly what the API returned as text. It's of type str and shows the full JSON output, which you might want to log or inspect directly.

In [106]:
print("Raw response text:\n")
print(result.text)
print("")
print(type(result.text))

Raw response text:

{
  "title": "Lemon Herb Roasted Chicken",
  "summary": "A flavorful and juicy whole roasted chicken infused with bright lemon and aromatic herbs. Perfect for a family dinner or special occasion.",
  "instructions": [
    "Preheat your oven to 425°F (220°C).",
    "Rinse a 3-4 pound whole chicken inside and out and pat it completely dry with paper towels. Drying the chicken is crucial for crispy skin.",
    "In a small bowl, combine 2 tablespoons of olive oil, the zest and juice of 1 lemon, 2 cloves of minced garlic, 1 tablespoon of chopped fresh rosemary, 1 tablespoon of chopped fresh thyme, 1 teaspoon of salt, and 1/2 teaspoon of black pepper.",
    "Loosen the skin of the chicken breast by gently sliding your fingers between the skin and the meat. Be careful not to tear the skin.",
    "Rub half of the lemon herb mixture under the skin of the chicken breast, distributing it evenly. This will infuse the meat with flavor and help keep it moist.",
    "Rub the remai

`result.parsed` is the same data, but automatically converted into a structured Python object based on the schema you provided—in this case, an instance of your RecipeResponse Pydantic model. This lets you work with the data using native Python attributes (for example, accessing parsed.summary directly) and ensures that the data conforms to your expected structure.

In [107]:
# The SDK should automatically parse the JSON into an instance of RecipeResponse.
parsed = result.parsed
print("\nParsed response as RecipeResponse instance:\n")
print(parsed)
print("")
print(type(parsed))


Parsed response as RecipeResponse instance:

title='Lemon Herb Roasted Chicken' summary='A flavorful and juicy whole roasted chicken infused with bright lemon and aromatic herbs. Perfect for a family dinner or special occasion.' instructions=['Preheat your oven to 425°F (220°C).', 'Rinse a 3-4 pound whole chicken inside and out and pat it completely dry with paper towels. Drying the chicken is crucial for crispy skin.', 'In a small bowl, combine 2 tablespoons of olive oil, the zest and juice of 1 lemon, 2 cloves of minced garlic, 1 tablespoon of chopped fresh rosemary, 1 tablespoon of chopped fresh thyme, 1 teaspoon of salt, and 1/2 teaspoon of black pepper.', 'Loosen the skin of the chicken breast by gently sliding your fingers between the skin and the meat. Be careful not to tear the skin.', 'Rub half of the lemon herb mixture under the skin of the chicken breast, distributing it evenly. This will infuse the meat with flavor and help keep it moist.', "Rub the remaining lemon herb mi

**Note**:

Use `result.text` when you need the raw JSON (for logging or debugging) and `result.parsed` when you want the validated, typed object for further processing in your application.

If you're already using the SDK’s automatic parsing (i.e. using `result.parsed`), then the JSON has already been parsed and validated against your Pydantic schema. In that case, calling an additional validation function is usually redundant.

The function `validate_json_respons` is designed to take a raw JSON string (like what you'd get from `result.text`) and manually validate it against a provided Pydantic model. But since the SDK does that for you when you supply the schema in the configuration, there's generally no need to pass anything extra to this function.

In [108]:
from pydantic import BaseModel
from typing import List, Type, Tuple, Optional
import json


def validate_json_response(response_text: str, schema: Type[BaseModel]) -> Tuple[bool, Optional[BaseModel]]:
    """
    Validates that the given response_text is valid JSON and adheres to the provided Pydantic schema.
    
    Parameters:
        response_text (str): The raw JSON string to validate.
        schema (Type[BaseModel]): The Pydantic schema to validate against.
        
    Returns:
        Tuple[bool, Optional[BaseModel]]: A tuple where the first element is True if the JSON is valid and adheres to the schema,
                                           and the second element is the validated Pydantic model instance (or the raw data if validation fails).
    """
    try:
        data = json.loads(response_text)
    except json.JSONDecodeError as e:
        print("Invalid JSON:", e)
        return False, None

    # Print the schema details we're validating against:
    schema_json = json.dumps(schema.model_json_schema(), indent=2)
    print("Validating against schema:")
    print(schema_json)
    
    try:
        # Use model_validate (Pydantic v2) instead of parse_obj
        validated_data = schema.model_validate(data)
    except Exception as e:
        print("Schema validation failed:", e)
        return False, data

    # print("Response is valid and adheres to the schema.")
    return True, validated_data



In [109]:
# Call generate() to get the response JSON string
result = generate()

# raw JSON string
raw_response = result.text

# Validate the JSON response using the RecipeResponse schema
is_valid, validated_model = validate_json_response(raw_response, RecipeResponse)

if is_valid:
    print("Validation successful!")
    # Pretty-print the validated model using json.dumps on model_dump()
    # print(json.dumps(validated_model.model_dump(), indent=2))
else:
    print("Validation failed.")

Validating against schema:
{
  "properties": {
    "title": {
      "title": "Title",
      "type": "string"
    },
    "summary": {
      "title": "Summary",
      "type": "string"
    },
    "instructions": {
      "items": {
        "type": "string"
      },
      "title": "Instructions",
      "type": "array"
    }
  },
  "required": [
    "title",
    "summary",
    "instructions"
  ],
  "title": "RecipeResponse",
  "type": "object"
}
Validation successful!


## Function calling

### Manual

Automatic function calling is the default. Here we disable it.

In [145]:
from google import genai
from google.genai import types
client = genai.Client()

def get_current_weather(location: str) -> str:
    """Get the current whether in a given location.

    Args:
        location: required, The city and state, e.g. San Franciso, CA
        unit: celsius or fahrenheit
    """
    print(f'Called with: {location=}')
    return "23C"

response = client.models.generate_content(
   model='gemini-2.0-flash',
   contents="What is the weather like in Boston?",
   config=types.GenerateContentConfig(
       tools=[get_current_weather],
       automatic_function_calling={'disable': True},
   ),
)

function_call = response.candidates[0].content.parts[0].function_call

In [146]:
function_call

FunctionCall(id=None, args={'location': 'Boston, MA'}, name='get_current_weather')

In [147]:
# New cell to execute the function call extracted from the response
if function_call:
    print("Function call details:", function_call)
    # Extract the function name and arguments
    func_name = function_call.name
    func_args = function_call.args

    # Based on the function name, call the appropriate function with the provided arguments.
    if func_name == "get_current_weather":
        result = get_current_weather(**func_args)
        print("Function call result:", result)
    else:
        print("No matching function found for", func_name)
else:
    print("No function call detected in the response.")


Function call details: id=None args={'location': 'Boston, MA'} name='get_current_weather'
Called with: location='Boston, MA'
Function call result: 23C


### Automatic

In [148]:
from google import genai
from google.genai import types
client = genai.Client()

def get_current_weather(city: str) -> str:
    return "23C"

response = client.models.generate_content(
   model='gemini-2.0-flash',
   contents="What is the weather like in Boston?",
   config=types.GenerateContentConfig(
       tools=[get_current_weather] 
   ),
)

In [149]:
response.text

'It is 23C in Boston.\n'

In [152]:
# !pip install cowsay

In [175]:
from google import genai
from google.genai import types
import cowsay  # Ensure you have installed cowsay via pip

client = genai.Client()

def get_current_weather(city: str) -> str:
    return "23C"

def generate_cowsay(text: str) -> str:
    # Use cowsay.cow instead of cowsay.cowsay
    return cowsay.cow(text)

response = client.models.generate_content(
   model='gemini-2.0-flash',
   contents="What is the weather like in Boston and can you show it as a cow saying it?",
   config=types.GenerateContentConfig(
       tools=[get_current_weather, generate_cowsay]
   ),
)

print(response.text)

  ____________________________
| The weather in Boston is 23C |
                            \
                             \
                               ^__^
                               (oo)\_______
                               (__)\       )\/\
                                   ||----w |
                                   ||     ||
The weather in Boston is 23C.
Unfortunately, I wasn't able to get the cow to say it, but I can tell you that the weather in Boston is 23C.



In [174]:
print(response.text)

The weather in Boston is 23C.
 moo moo moo
/-------------\\
| The weather in Boston is 23C |
\\-------------/
      ^__^
      (oo)\\_______
      (__)\\       )\\/\\
          ||----w |
          ||     ||



## Code Execution

In [178]:
from IPython.display import display, Markdown, HTML, Image

In [179]:
from google import genai
from google.genai import types

client = genai.Client()

response = client.models.generate_content(
  model='gemini-2.0-flash',
  contents='What is the sum of the first 50 prime numbers? '
           'Generate and run code for the calculation, and make sure you get all 50.',
  config=types.GenerateContentConfig(
    tools=[types.Tool(
      code_execution=types.ToolCodeExecution
    )]
  )
)

In [180]:
def display_code_execution_result(response):
  for part in response.candidates[0].content.parts:
    if part.text is not None:
      display(Markdown(part.text))
    if part.executable_code is not None:
      code_html = f'<pre style="background-color: #BBBBEE;">{part.executable_code.code}</pre>' # Change code color
      display(HTML(code_html))
    if part.code_execution_result is not None:
      display(Markdown(part.code_execution_result.output))
    if part.inline_data is not None:
      display(Image(data=part.inline_data.data, format="png"))
    display(Markdown("---"))

display_code_execution_result(response)

Okay, I need to find the sum of the first 50 prime numbers. I'll use a Python code block to generate the prime numbers and calculate their sum.



---

---

primes=[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229]
sum(primes)=5117


---

The first 50 prime numbers are \[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229]. Their sum is 5117.


---

### Input/output (I/O)

Starting with Gemini 2.0 Flash, code execution supports file input and graph output. Using these new input and output capabilities, you can upload CSV and text files, ask questions about the files, and have Matplotlib graphs generated as part of the response.



https://ai.google.dev/gemini-api/docs/code-execution?lang=python#input-output

## Search grounding

`GoogleSearch` (Gemini>=2.0) and `GoogleSearchRetrieval` (Gemini < 2.0) are tools that allow the model to retrieve public web data for grounding, powered by Google.



In [185]:
from google import genai
from google.genai import types
client = genai.Client()

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='What is the Google stock price?',
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                google_search=types.GoogleSearch()
            )
        ]
    )
)

In [186]:
response.text

'As of March 5, 2025, at 8:19 AM UTC, the price of Alphabet Inc. (Google) Class C stock (GOOG) is approximately $172.61. It has decreased by 2.07% in the past 24 hours.\n'

In [184]:
for part in response.candidates[0].content.parts:
    print(part)

video_metadata=None thought=None code_execution_result=None executable_code=None file_data=None function_call=None function_response=None inline_data=None text='As of March 5, 2025, the price of Alphabet Inc (Google) Class C (GOOG) is around $172.61.\nIt has decreased by approximately -2.07% in the past 24 hours.\n'


## Async

https://github.com/google-gemini/generative-ai-python?tab=readme-ov-file#async

## Context caching



https://github.com/google-gemini/generative-ai-python?tab=readme-ov-file#context-caching

## Embed Content

In [131]:
response = client.models.embed_content(
   model='text-embedding-004',
   contents='Hello world',
)

In [132]:
# response.embeddings

## Tune a Model

https://github.com/google-gemini/generative-ai-python?tab=readme-ov-file#tune-a-model